This explains how I implemented the demosaicing algorithms and error analysis in DemosaicLab. I present the mathematical theory behind CFA sampling, present and explain the algorithms, and derive error bounds.


## 1. CFA Sampling

### 1.1 Bayer

The application takes a full-color RGB image and simulates what a camera sensor with a Bayer Color Filter Array (Bayer, 1976) would actually capture.

For an input image $I(x, y) = [R(x,y), G(x,y), B(x,y)]^T$, we define a sampling function $S: \mathbb{R}^{W \times H \times 3} \to \mathbb{R}^{W \times H}$ that extracts a single color value per pixel according to the CFA pattern:

$$M(x, y) = \begin{cases}
R(x, y) & \text{if } C(x, y) = \text{R} \\
G(x, y) & \text{if } C(x, y) = \text{G} \\
B(x, y) & \text{if } C(x, y) = \text{B}
\end{cases}$$

where $C(x, y)$ is the CFA pattern function.

For Bayer RGGB pattern:

$$C(x, y) = \begin{cases}
\text{R} & \text{if } x \bmod 2 = 0 \text{ and } y \bmod 2 = 0 \\
\text{G} & \text{if } x \bmod 2 \neq y \bmod 2 \\
\text{B} & \text{if } x \bmod 2 = 1 \text{ and } y \bmod 2 = 1
\end{cases}$$


### 1.2 X-Trans

The application also supports the X-Trans pattern (Fujifilm, 2012), which uses a 6×6 aperiodic pattern instead of the 2×2 periodic Bayer pattern. The X-Trans pattern is defined by a lookup table for the 6×6 base unit:

```
G R G G B G
B G B R G R
G R G G B G
G B G G R G
R G R B G B
G B G G R G
```

The pattern function $C(x, y)$ for X-Trans is:

$$C(x, y) = \text{XTransTable}[x \bmod 6, y \bmod 6]$$

where `XTransTable` is the 6×6 lookup table above. Unlike Bayer's simple modulo formula, X-Trans requires a lookup table because the pattern is aperiodic and designed to reduce moiré artifacts by breaking the 2×2 periodicity.

The implementation can be found in `simulateCFA()` in `cfa.ts` lines 34-61:
```typescript
export const simulateCFA = (
  imageData: ImageData, 
  type: CFAType, 
  layout: string = 'RGGB'
): Float32Array => {
  const { width, height, data } = imageData;
  const cfa = new Float32Array(width * height);
  
  let getChannel: (x: number, y: number) => 'r' | 'g' | 'b';
  
  if (type === 'bayer') {
    getChannel = getBayerKernel(layout);
  } else if (type === 'xtrans') {
    getChannel = getXTransKernel();
  }
  
  for (let y = 0; y < height; y++) {
    for (let x = 0; x < width; x++) {
      const ch = getChannel(x, y);
      const idx = (y * width + x) * 4;
      // Normalize 0-255 to 0-1
      const val = ch === 'r' ? data[idx] : ch === 'g' ? data[idx + 1] : data[idx + 2];
      cfa[y * width + x] = val / 255.0;
    }
  }
  
  return cfa;
};
```

## 2. Demosaicing Algorithms

### 2.1 Bilinear Interpolation

Bilinear interpolation interpolates missing channels by averaging available samples in local neighborhoods (Malvar et al., 2004). For any CFA pattern, the algorithm works as follows:

For missing channel $c$ at position $(x, y)$:

$$\hat{I}_c(x, y) = \frac{1}{|\mathcal{N}_c(x,y)|} \sum_{(x', y') \in \mathcal{N}_c(x,y)} M(x', y')$$

where $\mathcal{N}_c(x, y) = \{(x', y') : C(x', y') = c \text{ and } \|(x-x', y-y')\|_\infty \leq d\}$ is the set of neighboring pixels of color $c$ within distance $d$ (typically $d=1$ for Bayer, $d=2$ for X-Trans).

The algorithm:
1. For each pixel $(x, y)$ with sampled channel $C(x, y) = c_0$:
   - Set $\hat{I}_{c_0}(x, y) = M(x, y)$ (direct sample)
   - For each missing channel $c \neq c_0$:
     - Find all neighbors $(x', y')$ where $C(x', y') = c$ and $\|(x-x', y-y')\|_\infty \leq d$
     - Compute average: $\hat{I}_c(x, y) = \frac{1}{|\mathcal{N}_c(x,y)|} \sum_{(x', y') \in \mathcal{N}_c(x,y)} M(x', y')$

Bayer-specific example:

For the Bayer RGGB pattern with $d=1$, the three cases are:

1. Green at R/B pixel: Average 4 cross neighbors
   $$\hat{G}(x, y) = \frac{M(x-1,y) + M(x+1,y) + M(x,y-1) + M(x,y+1)}{4}$$

2. R/B at Green pixel: Average 2 horizontal or vertical neighbors
   $$\hat{R}(x, y) = \frac{M(x-1,y) + M(x+1,y)}{2} \text{ or } \frac{M(x,y-1) + M(x,y+1)}{2}$$

3. B at R pixel (or R at B): Average 4 diagonal neighbors
   $$\hat{B}(x, y) = \frac{M(x-1,y-1) + M(x+1,y-1) + M(x-1,y+1) + M(x+1,y+1)}{4}$$

For X-Trans with $d=2$ using the infinity norm $\|(x-x', y-y')\|_\infty \leq 2$, the neighborhood includes all pixels where $\max(|x-x'|, |y-y'|) \leq 2$, which forms a 5×5 window centered on the target pixel (2 pixels in each direction from the center).

Implementation (`demosaic.ts` lines 69-112):

```typescript
export const demosaicBayerBilinear = (
  input: DemosaicInput
): ImageData => {
  const { width, height, cfaData, cfaPatternMeta } = input;
  const output = new ImageData(width, height);
  const getChannel = getBayerKernel(cfaPatternMeta.layout);
  
  for (let y = 0; y < height; y++) {
    for (let x = 0; x < width; x++) {
      const centerVal = cfaData[y * width + x];
      const centerCh = getChannel(x, y);
      const idx = (y * width + x) * 4;
      
      let r = 0, g = 0, b = 0;
      const val = (cx: number, cy: number) => getCfaVal(cfaData, width, height, cx, cy);

      if (centerCh === 'g') {
        g = centerVal;
        const leftCh = getChannel(Math.max(0, x-1), y);
        if (leftCh === 'r' || getChannel(Math.min(width-1, x+1), y) === 'r') {
           r = (val(x-1, y) + val(x+1, y)) / 2;
           b = (val(x, y-1) + val(x, y+1)) / 2;
        } else {
           r = (val(x, y-1) + val(x, y+1)) / 2;
           b = (val(x-1, y) + val(x+1, y)) / 2;
        }
      } else if (centerCh === 'r') {
        r = centerVal;
        g = (val(x-1, y) + val(x+1, y) + val(x, y-1) + val(x, y+1)) / 4;
        b = (val(x-1, y-1) + val(x+1, y-1) + val(x-1, y+1) + val(x+1, y+1)) / 4;
      } else if (centerCh === 'b') {
        b = centerVal;
        g = (val(x-1, y) + val(x+1, y) + val(x, y-1) + val(x, y+1)) / 4;
        r = (val(x-1, y-1) + val(x+1, y-1) + val(x-1, y+1) + val(x+1, y+1)) / 4;
      }
      
      output.data[idx] = clamp(r);
      output.data[idx+1] = clamp(g);
      output.data[idx+2] = clamp(b);
      output.data[idx+3] = 255;
    }
  }
  return output;
};
```

### 2.2 Niu Edge Sensing (Niu et al., 2018)

The Niu algorithm uses edge-aware interpolation with a logistic function to weight different interpolation directions. For green interpolation at R/B pixels, it computes directional variations in four directions (horizontal, vertical, and two diagonals) and uses a logistic function to assign weights inversely proportional to variation.

The algorithm:
1. Computes directional variations: $\Delta_H, \Delta_V, \Delta_{D1}, \Delta_{D2}$
2. Applies logistic function: $w_i = \sigma(\Delta_i) = \frac{1}{1 + e^{-k(\Delta_i - \theta)}}$
3. Normalizes weights inversely: $n_i = 1 - \frac{w_i}{\sum_j w_j}$
4. Performs weighted interpolation: $\hat{G} = \frac{\sum_i G_i \cdot n_i}{\sum_i n_i}$

For R/B channels, it uses color difference interpolation: $\hat{B} = \hat{G} + \text{avg}(B - G)$ at neighboring locations.

Implementation (`demosaic.ts` lines 213-318):

```typescript
export const demosaicBayerNiuEdgeSensing = (input: DemosaicInput, params?: DemosaicParams): ImageData => {
  const { width, height, cfaData, cfaPatternMeta } = input;
  const output = new ImageData(width, height);
  const getChannel = getBayerKernel(cfaPatternMeta.layout);
  const threshold = params?.niuLogisticThreshold ?? 0.1;
  
  const val = (cx: number, cy: number) => getCfaVal(cfaData, width, height, cx, cy);
  
  // First pass: Interpolate green channel at R/B pixels
  const greenInterp = new Float32Array(width * height);
  for (let y = 0; y < height; y++) {
    for (let x = 0; x < width; x++) {
      const centerCh = getChannel(x, y);
      const centerVal = cfaData[y * width + x];
      
      if (centerCh === 'g') {
        greenInterp[y * width + x] = centerVal;
      } else {
        // Interpolate green at R/B pixel using edge-aware method
        const vars = computeDirectionalVariations(cfaData, width, height, x, y, getChannel);
        
        // Compute edge weights using logistic function
        const wH = logisticFunction(vars.horizontal, threshold);
        const wV = logisticFunction(vars.vertical, threshold);
        const wD1 = logisticFunction(vars.diagonal1, threshold);
        const wD2 = logisticFunction(vars.diagonal2, threshold);
        
        // Normalize weights (inverse: lower variation = higher weight)
        const sumW = wH + wV + wD1 + wD2;
        const nH = (sumW > 0) ? (1.0 - wH / sumW) : 0.25;
        const nV = (sumW > 0) ? (1.0 - wV / sumW) : 0.25;
        const nD1 = (sumW > 0) ? (1.0 - wD1 / sumW) : 0.25;
        const nD2 = (sumW > 0) ? (1.0 - wD2 / sumW) : 0.25;
        const totalN = nH + nV + nD1 + nD2;
        
        // Weighted interpolation
        const gH = (val(x - 1, y) + val(x + 1, y)) / 2;
        const gV = (val(x, y - 1) + val(x, y + 1)) / 2;
        const gD1 = (val(x - 1, y - 1) + val(x + 1, y + 1)) / 2;
        const gD2 = (val(x - 1, y + 1) + val(x + 1, y - 1)) / 2;
        
        greenInterp[y * width + x] = (gH * nH + gV * nV + gD1 * nD1 + gD2 * nD2) / totalN;
      }
    }
  }
  
  // Second pass: Interpolate R/B channels
  for (let y = 0; y < height; y++) {
    for (let x = 0; x < width; x++) {
      const centerCh = getChannel(x, y);
      const centerVal = cfaData[y * width + x];
      const idx = (y * width + x) * 4;
      
      let r = 0, g = 0, b = 0;
      
      if (centerCh === 'r') {
        r = centerVal;
        g = greenInterp[y * width + x];
        
        // Interpolate blue using color difference (B - G)
        const bMinusG = [
          val(x - 1, y - 1) - greenInterp[(y - 1) * width + (x - 1)],
          val(x + 1, y - 1) - greenInterp[(y - 1) * width + (x + 1)],
          val(x - 1, y + 1) - greenInterp[(y + 1) * width + (x - 1)],
          val(x + 1, y + 1) - greenInterp[(y + 1) * width + (x + 1)]
        ];
        const avgBMinusG = bMinusG.reduce((a, b) => a + b, 0) / bMinusG.length;
        b = g + avgBMinusG;
      } else if (centerCh === 'b') {
        b = centerVal;
        g = greenInterp[y * width + x];
        
        // Interpolate red using color difference (R - G)
        const rMinusG = [
          val(x - 1, y - 1) - greenInterp[(y - 1) * width + (x - 1)],
          val(x + 1, y - 1) - greenInterp[(y - 1) * width + (x + 1)],
          val(x - 1, y + 1) - greenInterp[(y + 1) * width + (x - 1)],
          val(x + 1, y + 1) - greenInterp[(y + 1) * width + (x + 1)]
        ];
        const avgRMinusG = rMinusG.reduce((a, b) => a + b, 0) / rMinusG.length;
        r = g + avgRMinusG;
      } else {
        // Green pixel
        g = centerVal;
        
        const leftCh = getChannel(Math.max(0, x - 1), y);
        if (leftCh === 'r' || getChannel(Math.min(width - 1, x + 1), y) === 'r') {
          r = (val(x - 1, y) + val(x + 1, y)) / 2;
          b = (val(x, y - 1) + val(x, y + 1)) / 2;
        } else {
          r = (val(x, y - 1) + val(x, y + 1)) / 2;
          b = (val(x - 1, y) + val(x + 1, y)) / 2;
        }
      }
      
      output.data[idx] = clamp(r);
      output.data[idx + 1] = clamp(g);
      output.data[idx + 2] = clamp(b);
      output.data[idx + 3] = 255;
    }
  }
  
  return output;
};
```

### 2.3 Lien Edge-Based (Lien et al., 2017)

The Lien algorithm detects edge direction by comparing horizontal and vertical color differences, then interpolates perpendicular to the detected edge direction.

For green interpolation at R/B pixels:
- Compute $\Delta_H = |I(x-1,y) - I(x+1,y)|$ and $\Delta_V = |I(x,y-1) - I(x,y+1)|$
- If $\Delta_H < \Delta_V$ (horizontal edge), interpolate vertically: $\hat{G} = \frac{G(x,y-1) + G(x,y+1)}{2}$
- Otherwise (vertical edge), interpolate horizontally: $\hat{G} = \frac{G(x-1,y) + G(x+1,y)}{2}$

For R/B channels, it uses color difference interpolation similar to Niu.

Implementation (`demosaic.ts` lines 321-419):

```typescript
export const demosaicBayerLienEdgeBased = (input: DemosaicInput): ImageData => {
  const { width, height, cfaData, cfaPatternMeta } = input;
  const output = new ImageData(width, height);
  const getChannel = getBayerKernel(cfaPatternMeta.layout);
  
  const val = (cx: number, cy: number) => getCfaVal(cfaData, width, height, cx, cy);
  
  for (let y = 0; y < height; y++) {
    for (let x = 0; x < width; x++) {
      const centerCh = getChannel(x, y);
      const centerVal = cfaData[y * width + x];
      const idx = (y * width + x) * 4;
      
      let r = 0, g = 0, b = 0;
      
      if (centerCh === 'g') {
        g = centerVal;
        
        // Detect edge direction using simple color differences
        const diffH = Math.abs(val(x - 1, y) - val(x + 1, y));
        const diffV = Math.abs(val(x, y - 1) - val(x, y + 1));
        
        const leftCh = getChannel(Math.max(0, x - 1), y);
        if (leftCh === 'r' || getChannel(Math.min(width - 1, x + 1), y) === 'r') {
          // Red row
          if (diffH < diffV) {
            // Horizontal edge - interpolate along vertical
            r = (val(x - 1, y) + val(x + 1, y)) / 2;
            b = (val(x, y - 1) + val(x, y + 1)) / 2;
          } else {
            // Vertical edge - interpolate along horizontal
            r = (val(x, y - 1) + val(x, y + 1)) / 2;
            b = (val(x - 1, y) + val(x + 1, y)) / 2;
          }
        } else {
          // Blue row
          if (diffH < diffV) {
            r = (val(x, y - 1) + val(x, y + 1)) / 2;
            b = (val(x - 1, y) + val(x + 1, y)) / 2;
          } else {
            r = (val(x - 1, y) + val(x + 1, y)) / 2;
            b = (val(x, y - 1) + val(x, y + 1)) / 2;
          }
        }
      } else if (centerCh === 'r') {
        r = centerVal;
        
        // Interpolate green using edge-aware method
        const diffH = Math.abs(val(x - 1, y) - val(x + 1, y));
        const diffV = Math.abs(val(x, y - 1) - val(x, y + 1));
        
        if (diffH < diffV) {
          // Prefer horizontal interpolation
          g = (val(x - 1, y) + val(x + 1, y)) / 2;
        } else {
          // Prefer vertical interpolation
          g = (val(x, y - 1) + val(x, y + 1)) / 2;
        }
        
        // Interpolate blue using color difference
        const avgB = (val(x - 1, y - 1) + val(x + 1, y - 1) + val(x - 1, y + 1) + val(x + 1, y + 1)) / 4;
        const avgG = (val(x - 1, y) + val(x + 1, y) + val(x, y - 1) + val(x, y + 1)) / 4;
        b = avgB + (g - avgG);
      } else {
        // Blue pixel
        b = centerVal;
        
        // Interpolate green using edge-aware method
        const diffH = Math.abs(val(x - 1, y) - val(x + 1, y));
        const diffV = Math.abs(val(x, y - 1) - val(x, y + 1));
        
        if (diffH < diffV) {
          g = (val(x - 1, y) + val(x + 1, y)) / 2;
        } else {
          g = (val(x, y - 1) + val(x, y + 1)) / 2;
        }
        
        // Interpolate red using color difference
        const avgR = (val(x - 1, y - 1) + val(x + 1, y - 1) + val(x - 1, y + 1) + val(x + 1, y + 1)) / 4;
        const avgG = (val(x - 1, y) + val(x + 1, y) + val(x, y - 1) + val(x, y + 1)) / 4;
        r = avgR + (g - avgG);
      }
      
      output.data[idx] = clamp(r);
      output.data[idx + 1] = clamp(g);
      output.data[idx + 2] = clamp(b);
      output.data[idx + 3] = 255;
    }
  }
  
  return output;
};
```

### 2.4 Wu Polynomial Interpolation (Wu et al., 2016)

The Wu algorithm uses polynomial interpolation with distance-weighted averaging. It fits a polynomial of degree $d$ to neighboring samples, weighting by inverse distance squared: $w_i = \frac{1}{1 + d_i^2}$.

For green interpolation:
$$\hat{G}(x, y) = \frac{\sum_i G_i \cdot w_i}{\sum_i w_i}$$

where $w_i = \frac{1}{1 + d_i^2}$ and $d_i$ is the distance to neighbor $i$.

For R/B channels, the algorithm uses color difference interpolation: $\hat{B} = \hat{G} + \text{avg}(B - G)$ at neighboring locations.

Implementation (`demosaic.ts` lines 422-562):

```typescript
export const demosaicBayerWuPolynomial = (input: DemosaicInput, params?: DemosaicParams): ImageData => {
  const { width, height, cfaData, cfaPatternMeta } = input;
  const output = new ImageData(width, height);
  const getChannel = getBayerKernel(cfaPatternMeta.layout);
  const degree = params?.wuPolynomialDegree ?? 2;
  
  const val = (cx: number, cy: number) => getCfaVal(cfaData, width, height, cx, cy);
  
  // First pass: Interpolate green channel
  const greenInterp = new Float32Array(width * height);
  for (let y = 0; y < height; y++) {
    for (let x = 0; x < width; x++) {
      const centerCh = getChannel(x, y);
      const centerVal = cfaData[y * width + x];
      
      if (centerCh === 'g') {
        greenInterp[y * width + x] = centerVal;
      } else {
        // Use polynomial interpolation for green
        const neighbors = [
          { v: val(x - 1, y), d: 1 },
          { v: val(x + 1, y), d: 1 },
          { v: val(x, y - 1), d: 1 },
          { v: val(x, y + 1), d: 1 }
        ];
        const values = neighbors.map(n => n.v);
        const positions = neighbors.map(n => n.d);
        greenInterp[y * width + x] = polynomialInterpolate(values, positions, degree);
      }
    }
  }
  
  // Second pass: Interpolate R/B using color difference
  for (let y = 0; y < height; y++) {
    for (let x = 0; x < width; x++) {
      const centerCh = getChannel(x, y);
      const centerVal = cfaData[y * width + x];
      const idx = (y * width + x) * 4;
      
      let r = 0, g = 0, b = 0;
      
      if (centerCh === 'r') {
        r = centerVal;
        g = greenInterp[y * width + x];
        
        // Interpolate blue using color difference (B - G)
        const bMinusG = [
          val(x - 1, y - 1) - greenInterp[(y - 1) * width + (x - 1)],
          val(x + 1, y - 1) - greenInterp[(y - 1) * width + (x + 1)],
          val(x - 1, y + 1) - greenInterp[(y + 1) * width + (x - 1)],
          val(x + 1, y + 1) - greenInterp[(y + 1) * width + (x + 1)]
        ];
        const avgBMinusG = bMinusG.reduce((a, b) => a + b, 0) / bMinusG.length;
        b = g + avgBMinusG;
      } else if (centerCh === 'b') {
        b = centerVal;
        g = greenInterp[y * width + x];
        
        // Interpolate red using color difference (R - G)
        const rMinusG = [
          val(x - 1, y - 1) - greenInterp[(y - 1) * width + (x - 1)],
          val(x + 1, y - 1) - greenInterp[(y - 1) * width + (x + 1)],
          val(x - 1, y + 1) - greenInterp[(y + 1) * width + (x - 1)],
          val(x + 1, y + 1) - greenInterp[(y + 1) * width + (x + 1)]
        ];
        const avgRMinusG = rMinusG.reduce((a, b) => a + b, 0) / rMinusG.length;
        r = g + avgRMinusG;
      } else {
        // Green pixel
        g = centerVal;
        
        const leftCh = getChannel(Math.max(0, x - 1), y);
        if (leftCh === 'r' || getChannel(Math.min(width - 1, x + 1), y) === 'r') {
          const rVals = [val(x - 1, y), val(x + 1, y)];
          const rPositions = [1, 1];
          const bVals = [val(x, y - 1), val(x, y + 1)];
          const bPositions = [1, 1];
          r = polynomialInterpolate(rVals, rPositions, degree);
          b = polynomialInterpolate(bVals, bPositions, degree);
        } else {
          const rVals = [val(x, y - 1), val(x, y + 1)];
          const rPositions = [1, 1];
          const bVals = [val(x - 1, y), val(x + 1, y)];
          const bPositions = [1, 1];
          r = polynomialInterpolate(rVals, rPositions, degree);
          b = polynomialInterpolate(bVals, bPositions, degree);
        }
      }
      
      output.data[idx] = clamp(r);
      output.data[idx + 1] = clamp(g);
      output.data[idx + 2] = clamp(b);
      output.data[idx + 3] = 255;
    }
  }
  
  return output;
};
```

### 2.5 Kiku Residual Interpolation (Kiku et al., 2016)

The Kiku algorithm performs iterative residual interpolation. It starts with a bilinear estimate, then iteratively refines by:
1. Computing residual at sampled locations: $R^{(k)}(x_s, y_s) = M(x_s, y_s) - \hat{I}^{(k)}(x_s, y_s)$
2. Interpolating the residual: $\hat{R}^{(k)} = \text{Interp}(R^{(k)})$
3. Updating estimate: $\hat{I}^{(k+1)} = \hat{I}^{(k)} + \hat{R}^{(k)}$

This process converges geometrically, reducing error by approximately half each iteration up to the fundamental limit of bilinear interpolation.

Implementation (`demosaic.ts` lines 565-693):

```typescript
export const demosaicBayerKikuResidual = (input: DemosaicInput, params?: DemosaicParams): ImageData => {
  const { width, height, cfaData, cfaPatternMeta } = input;
  const iterations = params?.kikuResidualIterations ?? 1;
  const getChannel = getBayerKernel(cfaPatternMeta.layout);
  
  const val = (cx: number, cy: number) => getCfaVal(cfaData, width, height, cx, cy);
  
  // Initial estimation using bilinear interpolation
  const initial = demosaicBayerBilinear(input);
  
  // Convert initial estimate to Float32Array for processing
  const initialR = new Float32Array(width * height);
  const initialG = new Float32Array(width * height);
  const initialB = new Float32Array(width * height);
  
  for (let i = 0; i < width * height; i++) {
    initialR[i] = initial.data[i * 4] / 255.0;
    initialG[i] = initial.data[i * 4 + 1] / 255.0;
    initialB[i] = initial.data[i * 4 + 2] / 255.0;
  }
  
  // Refine estimates: initial + interpolated residuals (with iterations)
  const residualR = new Float32Array(width * height);
  const residualG = new Float32Array(width * height);
  const residualB = new Float32Array(width * height);
  const interpolatedResidualR = new Float32Array(width * height);
  const interpolatedResidualG = new Float32Array(width * height);
  const interpolatedResidualB = new Float32Array(width * height);
  let currentR = new Float32Array(initialR);
  let currentG = new Float32Array(initialG);
  let currentB = new Float32Array(initialB);
  
  for (let iter = 0; iter < iterations; iter++) {
    // Compute residuals from current estimate
    for (let y = 0; y < height; y++) {
      for (let x = 0; x < width; x++) {
        const centerCh = getChannel(x, y);
        const centerVal = cfaData[y * width + x];
        const idx = y * width + x;
        
        if (centerCh === 'r') {
          residualR[idx] = centerVal - currentR[idx];
          residualG[idx] = 0;
          residualB[idx] = 0;
        } else if (centerCh === 'g') {
          residualR[idx] = 0;
          residualG[idx] = centerVal - currentG[idx];
          residualB[idx] = 0;
        } else {
          residualR[idx] = 0;
          residualG[idx] = 0;
          residualB[idx] = centerVal - currentB[idx];
        }
      }
    }
    
    // Interpolate residuals (reuse the same interpolation logic)
    for (let y = 0; y < height; y++) {
      for (let x = 0; x < width; x++) {
        const centerCh = getChannel(x, y);
        const idx = y * width + x;
        
        if (centerCh === 'r') {
          interpolatedResidualR[idx] = residualR[idx];
          interpolatedResidualG[idx] = (
            residualG[(y - 1) * width + x] +
            residualG[(y + 1) * width + x] +
            residualG[y * width + (x - 1)] +
            residualG[y * width + (x + 1)]
          ) / 4;
          interpolatedResidualB[idx] = (
            residualB[(y - 1) * width + (x - 1)] +
            residualB[(y - 1) * width + (x + 1)] +
            residualB[(y + 1) * width + (x - 1)] +
            residualB[(y + 1) * width + (x + 1)]
          ) / 4;
        } else if (centerCh === 'g') {
          interpolatedResidualG[idx] = residualG[idx];
          const leftCh = getChannel(Math.max(0, x - 1), y);
          if (leftCh === 'r' || getChannel(Math.min(width - 1, x + 1), y) === 'r') {
            interpolatedResidualR[idx] = (residualR[y * width + (x - 1)] + residualR[y * width + (x + 1)]) / 2;
            interpolatedResidualB[idx] = (residualB[(y - 1) * width + x] + residualB[(y + 1) * width + x]) / 2;
          } else {
            interpolatedResidualR[idx] = (residualR[(y - 1) * width + x] + residualR[(y + 1) * width + x]) / 2;
            interpolatedResidualB[idx] = (residualB[y * width + (x - 1)] + residualB[y * width + (x + 1)]) / 2;
          }
        } else {
          interpolatedResidualB[idx] = residualB[idx];
          interpolatedResidualG[idx] = (
            residualG[(y - 1) * width + x] +
            residualG[(y + 1) * width + x] +
            residualG[y * width + (x - 1)] +
            residualG[y * width + (x + 1)]
          ) / 4;
          interpolatedResidualR[idx] = (
            residualR[(y - 1) * width + (x - 1)] +
            residualR[(y - 1) * width + (x + 1)] +
            residualR[(y + 1) * width + (x - 1)] +
            residualR[(y + 1) * width + (x + 1)]
          ) / 4;
        }
      }
    }
    
    // Update current estimate
    for (let i = 0; i < width * height; i++) {
      currentR[i] += interpolatedResidualR[i];
      currentG[i] += interpolatedResidualG[i];
      currentB[i] += interpolatedResidualB[i];
    }
  }
  
  // Final output
  const output = new ImageData(width, height);
  for (let y = 0; y < height; y++) {
    for (let x = 0; x < width; x++) {
      const idx = y * width + x;
      const outIdx = (y * width + x) * 4;
      
      output.data[outIdx] = clamp(currentR[idx]);
      output.data[outIdx + 1] = clamp(currentG[idx]);
      output.data[outIdx + 2] = clamp(currentB[idx]);
      output.data[outIdx + 3] = 255;
    }
  }
  
  return output;
};
```

## 3. Error Analysis

### 3.1 Error Metrics

#### 3.1.1 Mean Squared Error (MSE)

MSE is defined as:

$$\text{MSE} = \frac{1}{3WH} \sum_{c \in \{R,G,B\}} \sum_{x=0}^{W-1} \sum_{y=0}^{H-1} (I_c(x,y) - \hat{I}_c(x,y))^2$$

Per-channel MSE:

$$\text{MSE}_c = \frac{1}{WH} \sum_{x,y} (I_c(x,y) - \hat{I}_c(x,y))^2 = \mathbb{E}[(I_c - \hat{I}_c)^2]$$

MSE is used because it's the $L_2$ norm squared, which:
1. Has clean mathematical properties (differentiable, convex)
2. Penalizes large errors more than small ones (due to squaring)
3. Relates to signal power and SNR in signal processing theory

The implementation can be found in `computeErrorStats()` in `demosaic.ts` lines 309-409.

#### 3.1.2 Peak Signal-to-Noise Ratio (PSNR)

PSNR is defined as (Gunturk et al., 2005):

$$\text{PSNR} = 10 \log_{10}\left(\frac{\text{MAX}^2}{\text{MSE}}\right) = 20 \log_{10}\left(\frac{\text{MAX}}{\sqrt{\text{MSE}}}\right)$$

For 8-bit images, $\text{MAX} = 255$:

$$\text{PSNR} = 20 \log_{10}(255) - 10\log_{10}(\text{MSE}) \approx 48.13 - 10\log_{10}(\text{MSE})$$

PSNR can be interpreted as:

$$\text{PSNR}(\text{dB}) = \begin{cases}
> 40 & \text{Excellent - artifacts barely visible} \\
30-40 & \text{Good - minor artifacts} \\
20-30 & \text{Fair - noticeable artifacts} \\
< 20 & \text{Poor - severe artifacts}
\end{cases}$$

Why logarithmic? Human perception of image quality is approximately logarithmic with respect to error power (Gunturk et al., 2005). A 10 dB difference roughly corresponds to a perceivable quality difference.

#### 3.1.3 Structural Similarity Index (SSIM)

MSE and PSNR treat all errors equally, but human vision is sensitive to structural information (patterns, edges, textures) more than absolute pixel values (Wang et al., 2004). For two image patches $\mathbf{x}$ and $\mathbf{y}$, SSIM is defined as (Wang et al., 2004):

$$\text{SSIM}(\mathbf{x}, \mathbf{y}) = \frac{(2\mu_x\mu_y + C_1)(2\sigma_{xy} + C_2)}{(\mu_x^2 + \mu_y^2 + C_1)(\sigma_x^2 + \sigma_y^2 + C_2)}$$

where:
- $\mu_x = \frac{1}{N}\sum_{i=1}^N x_i$ (mean)
- $\sigma_x^2 = \frac{1}{N-1}\sum_{i=1}^N (x_i - \mu_x)^2$ (variance)
- $\sigma_{xy} = \frac{1}{N-1}\sum_{i=1}^N (x_i - \mu_x)(y_i - \mu_y)$ (covariance)
- $C_1 = (0.01 \times 255)^2$, $C_2 = (0.03 \times 255)^2$ (stability constants)

SSIM measures three components:

1. Luminance comparison: $l(\mathbf{x}, \mathbf{y}) = \frac{2\mu_x\mu_y + C_1}{\mu_x^2 + \mu_y^2 + C_1}$
2. Contrast comparison: $c(\mathbf{x}, \mathbf{y}) = \frac{2\sigma_x\sigma_y + C_2}{\sigma_x^2 + \sigma_y^2 + C_2}$
3. Structure comparison: $s(\mathbf{x}, \mathbf{y}) = \frac{\sigma_{xy} + C_2/2}{\sigma_x\sigma_y + C_2/2}$

Then: $\text{SSIM} = l(\mathbf{x}, \mathbf{y}) \cdot c(\mathbf{x}, \mathbf{y}) \cdot s(\mathbf{x}, \mathbf{y})$

SSIM has the following properties (Wang et al., 2004):
- $\text{SSIM} \in [-1, 1]$, where 1 means perfect structural similarity
- Symmetric: $\text{SSIM}(\mathbf{x}, \mathbf{y}) = \text{SSIM}(\mathbf{y}, \mathbf{x})$
- More robust to uniform brightness/contrast shifts than MSE



### 3.2 Bilinear Interpolation Error Analysis

In this section we analyze the reconstruction error of bilinear interpolation given CFA samples, i.e.

$$e_c(x,y) := I^{\text{CFA}}_c(x,y) - \hat I_c(x,y),$$

where $I^{\text{CFA}}_c$ is the (subsampled) channel value implied by the CFA at $(x,y)$ and $\hat I_c$ is the value reconstructed by the algorithm. We do not model the aliasing error introduced by the CFA sampling itself; all bounds here are conditional on the CFA samples and characterize only the interpolation component of the error.

We assume each color channel $I_c(x,y)$ is sufficiently smooth and satisfies

$$|\nabla I_c|_\infty \le L,\qquad |H I_c|_\infty \le M,$$

where $H I_c$ is the Hessian (matrix of second derivatives) and the norms are taken over the image domain. Intuitively, $L$ bounds local gradients and $M$ bounds local curvature.

#### 3.2.1 Provable Worst-Case Bounds

Because bilinear interpolation is a convex combination of neighbor samples, the reconstructed value lies between the minimum and maximum input values. For 8-bit images with values in $[0,255]$:

$$|I_c(x,y) - \hat I_c(x,y)| \le 255.$$

At each Bayer pixel, one color channel is directly sampled (zero reconstruction error) and two are interpolated. Thus, per pixel:

* L1 total error

  $$|R_{\text{err}}| + |G_{\text{err}}| + |B_{\text{err}}| \le 2\cdot 255 = 510.$$

* L2 total error

  $$\sqrt{R_{\text{err}}^2 + G_{\text{err}}^2 + B_{\text{err}}^2} \le 255\sqrt{2} \approx 360.$$

Relating this to PSNR, for a single channel:

$$PSNR = 10\log_{10}\frac{255^2}{MSE} = 20\log_{10}\frac{255}{RMSE}.$$

If the error is 255 at every pixel, then $MSE=255^2$ and $PSNR = 0$ dB. This is the absolute worst possible case and is independent of the interpolation algorithm.

These bounds are provable and sharp but essentially trivial; they give no insight into how interpolation behaves on typical smooth natural images.

#### 3.2.2 Average Bounds (Smooth Images via Taylor Expansion)

For smooth images, we can derive more informative curvature-based bounds. Consider a point $(x,y)$ and a nearby point $(x',y')$ at distance $d = |(x'-x, y'-y)|$. Taylor expansion gives

$$I_c(x', y') = I_c(x, y) + \nabla I_c(x, y)^\top (x'-x, y'-y) + \frac{1}{2}(x'-x, y'-y)^\top H (x'-x, y'-y) + O(d^3).$$

Bilinear interpolation on a regular grid:

* Exactly reproduces constant and linear terms $(1, x, y)$,
* Exactly reproduces the mixed term $xy$,
* So the leading error comes from the pure second-derivative terms $I_{xx}$ and $I_{yy}$.

If the maximum distance from an interpolated pixel to the neighbors used in bilinear interpolation is $d$, then, using $|H I_c|_\infty \le M$, we obtain a conservative bound of the form

$$|I_c(x,y) - \hat I_c(x,y)| \le \frac{M d^2}{2} \equiv E.$$

Assuming (pessimistically) this bound holds with equality at every pixel:

$$MSE_c \le E^2,\qquad PSNR_c \ge 20\log_{10}\frac{255}{E} = 20\log_{10}\left(\frac{510}{M d^2}\right).$$

For a Bayer CFA, the farthest same-color neighbors used in simple bilinear demosaicing are typically at distance $d = \sqrt{2}$, giving $d^2 = 2$ and

$$E = \frac{M \cdot 2}{2} = M,\qquad PSNR_c \ge 20\log_{10}\left(\frac{255}{M}\right).$$

Illustrative bounds (per channel):

| Image Type     | Typical $M$ | Error Bound (Bayer, $d=\sqrt{2}$) | $MSE_c \le$ | $PSNR_c \ge$ |
| -------------- | ----------: | --------------------------------- | ----------: | -----------: |
| Smooth sky     |          ~1 | ≤ 1                               |           1 |        48 dB |
| Gentle texture |         ~10 | ≤ 10                              |         100 |        28 dB |
| Moderate edges |         ~50 | ≤ 50                              |        2500 |        14 dB |
| Sharp edges    |        ~200 | ≤ 200                             |       40000 |         2 dB |

These PSNR values are lower bounds under a pessimistic assumption that every pixel attains the maximum local error. In real images, only some regions have large curvature, so actual PSNR is typically higher.

These curvature-based bounds explain why bilinear works reasonably well on smooth regions (small $M$) but say nothing about aliasing or extreme patterns. For 8-bit images, the maximum possible discrete second derivative is on the order of 510 (e.g. checkerboard patterns), so for worst-case signals $M$ approaches this theoretical limit and the Taylor smoothness assumption itself breaks down. In that regime, the range-based bound $(|I_c - \hat I_c|\le 255)$ is the only meaningful guarantee.




### 3.3 Niu Edge-Sensing Bound

The Niu algorithm interpolates green using a weighted combination of directional estimates:

$$\hat G(x,y) = \frac{\sum_{i\in\{H,V,D1,D2\}} w_i, G_i}{\sum_i w_i},$$

where each $G_i$ is a directional interpolation (horizontal, vertical, and two diagonals), and weights are

$$w_i = 1 - \frac{\sigma(\Delta_i)}{\sum_j \sigma(\Delta_j)},\qquad \sigma(\Delta) = \frac{1}{1 + e^{-k(\Delta - \theta)}}.$$

Each neighbor $G_i$ can be expanded as

$$G_i = I_c(x,y) + \nabla I_c(x,y)^\top \vec d_i + \frac{1}{2}\vec d_i^\top H \vec d_i + O(|\vec d_i|^3),$$

where $\vec d_i$ is the offset from $(x,y)$ to the neighbor used for $G_i$. Substituting into the weighted average:

$$\hat G(x,y) = I_c(x,y) + \frac{\sum_i w_i, \nabla I_c^\top \vec d_i}{\sum_i w_i} + \frac{\sum_i w_i, \frac{1}{2}\vec d_i^\top H \vec d_i}{\sum_i w_i} + O(d_{\max}^3),$$

with $d_{\max} = \max_i |\vec d_i|$.

Unlike symmetric bilinear interpolation, the weights $w_i$ are generally asymmetric, so the linear term does not cancel in general. Using $|\nabla I_c|_\infty \le L$ and $|H I_c|_\infty \le M$, we obtain:

* Linear term bound

  $$\left| \frac{\sum_i w_i, \nabla I_c^\top \vec d_i}{\sum_i w_i} \right| \le L, d_{\max} ,\max_i \frac{w_i}{\sum_j w_j}.$$

* Quadratic term bound

  $$\left| \frac{\sum_i w_i, \frac{1}{2}\vec d_i^\top H \vec d_i}{\sum_i w_i} \right| \le \frac{M d_{\max}^2}{2}.$$

If edge detection correctly de-emphasizes across-edge directions, the largest normalized weight can be significantly less than 1. As a simple illustrative case for a Bayer CFA with $d_{\max} = \sqrt{2}$, suppose that good edge detection effectively halves the largest normalized weight, i.e.

$$\max_i \frac{w_i}{\sum_j w_j} \approx 0.5.$$

Then the reconstruction error magnitude is bounded by

$$|e_c(x,y)| \lesssim \frac{M d_{\max}^2}{2} + 0.5,L d_{\max} = M + 0.5 L\sqrt{2}.$$

Assuming this bound everywhere gives

$$MSE_c \le \bigl(M + 0.5 L\sqrt{2}\bigr)^2,\qquad PSNR_c \ge 20\log_{10}\frac{255}{M + 0.5 L\sqrt{2}}.$$

Example for a smooth sky with mild edges $(M\approx 1, L\approx 5)$:

$$|e_c| \lesssim 1 + 0.5\cdot 5\sqrt{2} \approx 4.5,\qquad PSNR_c \gtrsim 35,\text{dB}.$$

This bound is most informative when edges are well-detected and gradients are moderate. If edge detection fails (e.g. in textured or noisy regions), the asymmetric weights may amplify linear-gradient contributions, degrading performance toward or below bilinear levels.



### 3.4 Lien Edge-Based Bound

Lien's method detects edge orientation via local intensity differences:

* Horizontal variation:

  $$\Delta_H = |I(x-1,y) - I(x+1,y)|$$

* Vertical variation:

  $$\Delta_V = |I(x,y-1) - I(x,y+1)|.$$

If $\Delta_H < \Delta_V$ (horizontal edge), it interpolates along the smoother vertical direction (distance $d=1$):

$$\hat C(x,y) = \frac{C(x,y-1) + C(x,y+1)}{2}.$$

With unit pixel spacing, a Taylor expansion in the vertical direction gives

$$C(x,y\pm 1) = C(x,y) \pm C_y(x,y) + \frac{1}{2}C_{yy}(x,y) + O(1),$$

so the symmetric average yields

$$\hat C(x,y) = C(x,y) + \frac{1}{2}C_{yy}(x,y) + O(1).$$

The linear terms cancel, and the leading error is proportional to the second derivative. Using $|C_{yy}|\le M$ and ignoring the higher-order remainder in a conservative way:

* Correct classification (interpolating perpendicular to the edge, $d=1$):

  $$|C(x,y) - \hat C(x,y)| \lesssim \frac{M}{2}.$$

If the edge is misclassified and interpolation is done along the edge using diagonally separated pixels at distance $d = \sqrt{2}$, the curvature term scales like $d^2$, giving:

* Misclassification / diagonal interpolation:

  $$|C(x,y) - \hat C(x,y)| \lesssim \frac{M d^2}{2} = M.$$

For correct classification, assuming the error bound holds everywhere:

$$MSE_c \le \left(\frac{M}{2}\right)^2 = \frac{M^2}{4},\qquad PSNR_c \ge 20\log_{10}\frac{255}{M/2} = 20\log_{10}\frac{510}{M}.$$

Examples:

* Smooth sky, $M\approx 1$:  $E\approx 0.5$,  $PSNR_c \gtrsim 54$ dB.
* Gentle texture, $M\approx 10$: $E\approx 5$,  $PSNR_c \gtrsim 34$ dB.

The bound shows that Lien can be significantly better than bilinear (which has error $(\sim M)$) when edge direction is correctly identified and interpolation is performed along the smoother direction. In regions where directional estimates are unreliable (e.g. weak edges or ambiguous gradients), misclassification pushes the scheme back toward bilinear-like behavior.



### 3.5 Wu Polynomial Bound

Wu's method uses distance-based weighting:

$$w_i = \frac{1}{1 + d_i^2},\qquad \hat C(x,y) = \frac{\sum_i w_i, C_i}{\sum_i w_i},$$

where $C_i$ is the value at neighbor $i$ and $d_i$ is its distance from $(x,y)$.

Expanding each neighbor:

$$C_i = C(x,y) + \nabla C(x,y)^\top \vec d_i + \frac{1}{2}\vec d_i^\top H \vec d_i + O(|\vec d_i|^3),$$

we obtain

$$\hat C(x,y) = C(x,y) + \frac{\sum_i w_i, \nabla C^\top \vec d_i}{\sum_i w_i} + \frac{\sum_i w_i, \frac{1}{2}\vec d_i^\top H \vec d_i}{\sum_i w_i} + O(d_{\max}^3).$$

For a Bayer pattern, a common case is using four diagonal neighbors at equal distance $d=\sqrt{2}$. In that configuration:

* All $w_i$ are equal $(w_i = 1/(1+d^2) = 1/3)$,
* The diagonal offsets are symmetric and sum to zero,
* Therefore $\sum_i w_i \vec d_i = 0$, and the linear term cancels exactly, as in bilinear.

The leading error is again governed by curvature:

$$\left| \frac{\sum_i w_i, \frac{1}{2}\vec d_i^\top H \vec d_i}{\sum_i w_i} \right| \le \frac{M d_{\max}^2}{2}.$$

For $d_{\max}=\sqrt{2}$ this gives

$$|C(x,y) - \hat C(x,y)| \lesssim \frac{M\cdot 2}{2} = M.$$

Thus, under only the smoothness assumption $|H C|_\infty \le M$, Wu's scheme shares essentially the same worst-case reconstruction bound as bilinear in this configuration. The distance weighting improves empirical performance by downweighting farther (and often less reliable) neighbors, but it does not, by itself, tighten the uniform curvature-based error bound.

Assuming the bound holds everywhere:

$$MSE_c \le M^2,\qquad PSNR_c \ge 20\log_{10}\frac{255}{M}.$$

For $M=10$, this gives $PSNR_c \gtrsim 28$ dB, matching the bilinear curvature bound.

**Note on implementation**: This bound assumes a search radius of 1 (neighbors at distance $d \le 1$), which ensures symmetric neighbor configurations. However, with radius=1, all neighbors are equidistant, which eliminates the distance weighting benefit. The implementation uses radius=2 for Bayer to enable distance weighting: with radius=2, neighbors at different distances (e.g., $d=1$, $d=\sqrt{2}$, $d=2$, $d=\sqrt{5}$) allow closer neighbors to be weighted more heavily, improving edge handling. The theoretical bound (≥28 dB) remains valid for smooth images with radius=1, but practical performance on edges and natural images benefits from radius=2's distance weighting.

**Empirical observations**: On smooth synthetic images (Color Sweep, Color Patches), Wu achieves excellent performance (57.22 dB and 29.64 dB respectively), exceeding the bound. On natural images and edge patterns, radius=2 enables distance weighting to emphasize closer neighbors, improving performance compared to radius=1 which reduces to simple averaging.



### 3.6 Kiku Residual Bound

Kiku's method applies iterative residual interpolation:

1. Initial estimate via bilinear interpolation:

   $$\hat I^{(0)}.$$

2. For iteration $k = 0,\dots,K-1$:

   * Compute residual at CFA-sampled pixels:

     $$R^{(k)} = I^{\text{CFA}} - \hat I^{(k)}.$$

   * Interpolate the residual:

     $$\hat R^{(k)} = \text{Interp}(R^{(k)}).$$

   * Update the estimate:

     $$\hat I^{(k+1)} = \hat I^{(k)} + \hat R^{(k)}.$$

The first step has an error bounded by the bilinear curvature bound:

$$|I^{\text{CFA}} - \hat I^{(0)}| \lesssim \frac{M d^2}{2}.$$

If the residual $R^{(k)}$ becomes progressively smoother (smaller effective curvature) as $k$ increases, the same type of interpolation produces smaller and smaller errors. Empirically, this often behaves like geometric error decay:

$$|e^{(k+1)}| \approx \alpha ,|e^{(k)}|,\qquad 0 < \alpha < 1,$$

where $e^{(k)} = I^{\text{CFA}} - \hat I^{(k)}$ and $\alpha$ depends on image statistics and the interpolation operator. For typical natural images, a value $\alpha \approx 0.6$ is plausible.

After $K$ iterations,

$$|e^{(K)}| \approx \alpha^K |e^{(0)}| \lesssim \alpha^K \frac{M d^2}{2}.$$

For a Bayer pattern with $d = \sqrt{2}$ and $K=3$ iterations, taking $\alpha \approx 0.6$ yields

$$|e_c(x,y)| \lesssim 0.22M.$$

Assuming this holds everywhere:

$$MSE_c \lesssim (0.22 M)^2,\qquad PSNR_c \gtrsim 20\log_{10}\frac{255}{0.22 M} = 20\log_{10}\frac{1159}{M}.$$

For gentle texture with $M \approx 10$:

$$|e_c| \lesssim 2.2,\qquad PSNR_c \gtrsim 41,\text{dB}.$$

**Note on implementation**: The default implementation uses $K=1$ iteration. For $K=1$, the error bound is:

$$|e_c(x,y)| \lesssim \alpha \cdot \frac{M d^2}{2} \approx 0.6 \cdot M \approx 0.6M.$$

For $M \approx 10$:

$$|e_c| \lesssim 6,\qquad PSNR_c \gtrsim 20\log_{10}\frac{255}{6} \approx 32.6,\text{dB}.$$

This matches empirical observations where Kiku with $K=1$ achieves approximately 29 dB, slightly better than bilinear's 28 dB baseline.

Unlike previous bounds, this is explicitly empirical: it depends on the observed residual smoothness and is not a strict worst-case guarantee. Nevertheless, it captures the key idea that iterative residual interpolation can significantly reduce reconstruction error when the residual is smoother (less curved) than the original image.



### 3.7 Comparison of Bounds

For smooth images satisfying $|H I_c|_\infty \le M$ and $|\nabla I_c|_\infty \le L$, the reconstruction error bounds derived above can be summarized (per channel) as follows.

Using a representative case $(M = 10)$, $(L = 50)$:

| Algorithm  | Error Bound (per channel) | Conditions | $PSNR_c$ ($M=10$, $L=50$) |
| ---------- | ------------------------- | ---------- | ------------------------- |
| Bilinear   | $\|e_c\| \lesssim M$                  | Always                            | ≥ 28 dB    |
| Niu        | $\|e_c\| \lesssim M + 0.5 L\sqrt{2}$  | Good edge detection, moderate $L$ | ≥ 15 dB    |
| Lien       | $\|e_c\| \lesssim \frac{M}{2}$ to $M$ | Correct / incorrect direction     | ≈ 34–28 dB |
| Wu         | $\|e_c\| \lesssim M$                  | Smooth images, radius=1 (bound) / radius=2 (implementation) | ≥ 28 dB    |
| Kiku (K=1) | $\|e_c\| \lesssim 0.6M$               | Residual smooth, α≈0.6, default    | ≳ 33 dB    |
| Kiku (K=3) | $\|e_c\| \lesssim 0.22M$               | Residual smooth, α≈0.6, 3 iterations| ≳ 41 dB    |

Key takeaways:

* Bilinear provides a simple baseline with error proportional to local curvature $(M)$.

* Niu uses directional, asymmetric weights; when edge detection is accurate and gradients are modest, it can reduce across-edge blur, but the asymmetric weights introduce a linear-gradient term that can dominate in high-$L$ regions, making the worst-case bound loose and sometimes worse than bilinear.

* Lien achieves the tightest curvature-based bound when the edge direction is correctly identified, effectively reducing the interpolation distance from $\sqrt{2}$ to 1. Misclassification degrades performance back toward bilinear.

* Wu's distance weighting improves empirical behavior but, under the pure curvature bound, shares the same worst-case scaling as bilinear in symmetric configurations.

* Kiku offers strong empirical performance by iteratively refining the residual. The default implementation uses $K=1$ iteration, achieving approximately 29 dB (slightly better than bilinear). With $K=3$ iterations, the bound predicts ≳41 dB, but this requires multiple iterations which increases computational cost. The bound relies on observed geometric error decay and thus is not a strict worst-case guarantee.

All of these reconstruction bounds share the same fundamental limitation noted in §3.2.2: they assume smoothness $(M \ll 510)$ and do not account for aliasing introduced by CFA sampling itself. For pathological high-frequency patterns where discrete second derivatives approach their theoretical maximum, the Taylor-based smoothness assumptions break down and only the trivial range-based bound (255 per channel) remains valid.

# References

Alleysson, D., Susstrunk, S., & Hérault, J. (2005). Linear demosaicing inspired by the human visual system. IEEE Transactions on Image Processing, 14(4), 439-449. https://hal.science/hal-00204920/file/AlleyssonetalTIP05.pdf

Bayer, B. E. (1976). U.S. Patent No. 3,971,065. Washington, DC: U.S. Patent and Trademark Office.

Bae, T. W. (2020). Image-quality metric system for color filter array evaluation. Plos one, 15(5), e0232583. https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0232583

Dubois, E. (2005). Frequency-domain methods for demosaicking of Bayer-sampled color images. IEEE Signal Processing Letters, 12(12), 847-850. https://www.researchgate.net/profile/Eric-Dubois-6/publication/3343363_Frequency-domain_methods_for_demosaicking_of_Bayer-sampled_color_images/links/00463515ee52837cc6000000/Frequency-domain-methods-for-demosaicking-of-Bayer-sampled-color-images.pdf

Gunturk, B. K., Glotzbach, J., Altunbasak, Y., Schafer, R. W., & Mersereau, R. M. (2005). Demosaicking: color filter array interpolation. IEEE Signal Processing Magazine, 22(1), 44-54. https://www.ece.lsu.edu/gunturk/Topics/Demosaicking-1.pdf

Hirakawa, K., & Parks, T. W. (2005). Adaptive homogeneity-directed demosaicing algorithm. IEEE Transactions on Image Processing, 14(3), 360-369. http://elynxsdk.free.fr/ext-docs/Demosaicing/more/news0/Adaptive%20Homogeneity-Directed.pdf

Kimmel, R. (1999). Demosaicing: Image reconstruction from color CCD samples. IEEE Transactions on Image Processing, 8(9), 1221-1228. http://elynxsdk.free.fr/ext-docs/Demosaicing/more/Demosaicing-Image%20Reconstruction%20from%20Color%20CCD%20Samples.pdf

Li, X., Gunturk, B., & Zhang, L. (2008). Image demosaicing: A systematic survey. In Visual Communications and Image Processing 2008 (Vol. 6822, p. 68221J). International Society for Optics and Photonics. http://www4.comp.polyu.edu.hk/~cslzhang/paper/conf/demosaicing_survey.pdf

Malvar, H. S., He, L. W., & Cutler, R. (2004). High-quality linear interpolation for demosaicing of Bayer-patterned color images. In 2004 IEEE International Conference on Acoustics, Speech, and Signal Processing (Vol. 3, pp. iii-485). IEEE. https://home.cis.rit.edu/~cnspci/references/dip/demosaicking/malvar2004.pdf
 
Nyquist, H. (1928). Certain topics in telegraph transmission theory. Transactions of the American Institute of Electrical Engineers, 47(2), 617-644. https://www.academia.edu/download/60205560/1928_-_0505502420190805-3080-1thzpng.pdf

Rafinazari, M., & Dubois, E. (2014, October). Demosaicking algorithm for the Fujifilm X-Trans color filter array. In 2014 IEEE International Conference on Image Processing (ICIP) (pp. 660-663). IEEE.

Wang, Z., Bovik, A. C., Sheikh, H. R., & Simoncelli, E. P. (2004). Image quality assessment: from error visibility to structural similarity. IEEE Transactions on Image Processing, 13(4), 600-612. https://ece.uwaterloo.ca/~z70wang/publications/ssim.pdf


Niu, Y., Ouyang, J., Zuo, W., & Wang, F. (2018). Low cost edge sensing for high quality demosaicking. IEEE Transactions on Image Processing, 28(5), 2415–2427. Retrieved from https://arxiv.org/pdf/1806.00771


Wu, J., Anisetti, M., Wu, W., Damiani, E., & Jeon, G. (2016). Bayer demosaicking with polynomial interpolation. IEEE Transactions on Image Processing, 25(11), 5369–5382. Retrieved from https://air.unimi.it/bitstream/2434/440833/4/2434-440833.pdf


Lien, C.-Y., Yang, F.-J., & Chen, P.-Y. (2017). An efficient edge-based technique for color filter array demosaicking. IEEE Sensors Journal, 17(13), 4067–4074. Retrieved from https://ieeexplore.ieee.org/abstract/document/7931560/


Kiku, D., Monno, Y., Tanaka, M., & Okutomi, M. (2016). Beyond color difference: Residual interpolation for color image demosaicking. IEEE Transactions on Image Processing, 25(3), 1288–1300. Retrieved from https://ieeexplore.ieee.org/abstract/document/7383296/


Zhang, L., Wu, X., Buades, A., & Li, X. (2011). Color demosaicking by local directional interpolation and nonlocal adaptive thresholding. Journal of Electronic Imaging, 20(2), 023016. https://ira.lib.polyu.edu.hk/bitstream/10397/6039/1/Zhang_Color_Demosaicking_Local.pdf